# Insurance Medical
Identification of payer per the OOPS team for raven medical submits.

**Script**
* [scripts/cld/insurance_medical.ipynb](./scripts/cld/insurance_medical.ipynb)

**Prior Script(s)**
* [scripts/de/raven_provider.ipynb](./scripts/de/raven_provider.ipynb)
* [scripts/de/raven_patient.ipynb](./scripts/de/raven_patient.ipynb)

**Parameters**
* none

**Input**
* `de_raven_patient`
* `de_raven_payer`
  
**Output**
* `cld_med_ins`

**Review**
* [scripts/cld/insurance_medical.html](./scripts/cld/insurance_medical.html)

## Patrick requests
Below are a few requests/questions that Patrick Cronin has for this project

* What is the plan for maintaining the OOPS table (i.e.) what about penguin
* A standard way to import the data was included
* Move all reference tables to `RWD_REF.ANALYTICS`
* Need to understand how the remits work
* Where is the cleanup function (Omnya send Patrick script and he'll figure out how to)
* rwd_db.rwd_reference_library.insurance_types replaced rwd_db.rwd_reference_library.plan_type_determination

**Omnya To do**
* Send patrick Cleanup function


In [ ]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/conn/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications


# Data Setup
Pull necessary fields from `raven_patient` and `raven_payer` into a single table.

In [ ]:
%%read_sql

--Identify all data for claims needed for the analysis
DROP TABLE IF EXISTS cld_tmp_ins;
CREATE TRANSIENT TABLE cld_tmp_ins AS
    SELECT pat.patient_id, 
           pat.claim_id,
           pat.claim_number,
           pat.payer_sequence,
           Max(pat.member_adr_zip) AS member_adr_zip, 
           Max(pat.member_adr_state) AS member_adr_state, 
           Max(pat.year_of_service) AS year_of_service, 
           Max(pat.data_source) AS data_source, 
           Max(pay.payer_id) AS payer_id, 
           Max(pay.payer_name) AS payer_name, 
           Max(pay.type_coverage) AS type_coverage, 
           'S' AS claim_type
      FROM de_raven_patient pat
           JOIN de_raven_payer pay
             ON pat.patient_id = pay.patient_id
                AND pat.claim_id = pay.claim_id
                AND (pat.payer_sequence = pay.payer_sequence
                     OR pat.payer_sequence IS NULl)
                     
     GROUP BY pat.patient_id, pat.claim_number, pat.claim_id, pat.payer_sequence;
     
--Alter the table to add the other stuff that's needed for later updates
BEGIN;
ALTER TABLE cld_tmp_ins
    ADD ins_group varchar(100),
        ins_desc varchar(100),
        clean_raw_payer_name varchar(100),
        oops_payer_id varchar(100), 
        oops_payer_name varchar(100),
        oops_parent_id varchar(100),
        oops_parent_name varchar(100),
        oops_google_parent_id varchar(100),
        oops_google_parent_name varchar(100),
        update_payer varchar(100),
        oops_update varchar(100),
        multiple_payer_flag NUMERIC(1),
        impute_flag NUMERIC(1);
COMMIT;

In [ ]:
%%read_sql
--Review counts from the raw data
SELECT data_source, 
       Count(*) AS cnt,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins) AS pct
  FROM cld_tmp_ins
 GROUP BY data_source
  

# Set Insurance Type
Identify the insurance type for the claims from the standard DRG library `rwd_db.rwd_reference_library.insurance_types`

In [ ]:
%%read_sql
--Cleanup Payer Name
BEGIN;
UPDATE cld_tmp_ins
   SET clean_raw_payer_name = payer_name; --PRC Where is the cleanup funtion Cleanup(payer_name);
COMMIT;

--Update insurance type per the official DRG answer
BEGIN;
UPDATE cld_tmp_ins ins 
   SET ins.ins_group = type.ici_insurance_group,
       ins.ins_desc = type.insurance_type_description
  FROM rwd_db.rwd_reference_library.insurance_types type 
 WHERE ins.type_coverage = type.type_coverage;
COMMIT;

In [ ]:
%%read_sql
--Check the mapped coverage percentages
SELECT data_source,
       ins_group,
       Count(*) AS cnt,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins
                    WHERE data_source = ins.data_source) AS share,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins) AS pct
  FROM cld_tmp_ins ins
 GROUP BY data_source, ins_group
 ORDER BY data_source, cnt DESC

# Vulture Overrides
Update the Vulture claims from the vulture crosswalk

## Payer id & Name
Update the payer_id and payer_name from the crosswalk table

In [ ]:
%%read_sql
--Update Vulture Claims from the vulture crosswalk provided by the vendor
BEGIN;
UPDATE cld_tmp_ins ins
   SET ins.payer_id = xw.payer_id ,
       ins.payer_name = Coalesce(ins.payer_name,xw.payer_name),
       ins.update_payer = 'vulture_claim_ap_xwalk'
  FROM project_analytics.payer_mastering.vulture_claim_ap_xwalk xw
 WHERE Lower(ins.claim_number) = Lower('vul_' || xw.claim_id)
       AND data_source = 'VULTURE';
COMMIT;

In [ ]:
%%read_sql
--Reveiw counts of updated claims via vulture cross walk
--Around 75% of vulture claims were updated this way
SELECT data_source,
       update_payer,
       Count(*) AS row_cnt,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins
                    WHERE data_source = ins.data_source) as share,
        Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins) as pct             
  FROM cld_tmp_ins ins
 GROUP BY data_source, update_payer

## Medicare Override
Set all Vulture claims to medicare that have a numeric claim number

In [ ]:
%%read_sql
BEGIN;
UPDATE cld_tmp_ins 
   SET oops_payer_id = 'XW --157081941', 
       oops_payer_name = 'MEDICARE', 
       oops_parent_id = 'XW --157081941', 
       oops_parent_name = 'MEDICARE', 
       oops_google_parent_id = 'XW --157081941', 
       oops_google_parent_name = 'MEDICARE',
       oops_update = 'Vulture Medicare for numeric'
 WHERE Regexp_like(Replace(claim_number, 'vul_', '') , '[0-9]*') = true 
       AND To_number(Replace(claim_number, 'vul_', '')) BETWEEN 1 AND 147766537
       AND data_source = 'VULTURE';
COMMIT;

In [ ]:
%%read_sql
--Revew the row count updated by the vulture medicare override
SELECT data_source,
       oops_update,
       Count(*) AS row_cnt,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins
                    WHERE data_source = ins.data_source) as share,
        Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins) as pct             
  FROM cld_tmp_ins ins
 GROUP BY data_source, oops_update
 ORDER BY data_source,  oops_update

# OOPS Update
Update the OOPS data based on the oops reference table

In [ ]:
%%read_sql
--OOPS update for Albatross, Condor, and Vulture
BEGIN;
UPDATE cld_tmp_ins ins 
   SET ins.oops_payer_id = oops.oops_payer_id, 
       ins.oops_payer_name = oops.oops_payer_name, 
       ins.oops_parent_id = oops.oops_parent_id, 
       ins.oops_parent_name = oops.oops_parent_name, 
       ins.oops_google_parent_id = oops.oops_google_parent_id, 
       ins.oops_google_parent_name = oops.oops_google_parent_name,
       ins.oops_update = 'oops_payer_xwalk'
  FROM project_analytics.payer_mastering.oops_payer_xwalk oops
 WHERE ins.payer_id = oops.payerid
       AND ins.data_source IN ('ALBATROSS','CONDOR','VULTURE');
COMMIT;

In [ ]:
%%read_sql
--Revew the row count updated by the vulture medicare override
SELECT data_source,
       oops_update,
       Count(*) AS row_cnt,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins
                    WHERE data_source = ins.data_source) as share,
        Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins) as pct             
  FROM cld_tmp_ins ins
 GROUP BY data_source, oops_update
 ORDER BY data_source,  oops_update

# Impute OOPS
Impute the missing payers by identfying the nearest claim for that patient that has a legimate payer.

In [ ]:
%%read_sql
--Identify all of the claims without an OOPS payer
DROP TABLE IF EXISTS tmp_no_payer;
CREATE TRANSIENT TABLE tmp_no_payer AS
    SELECT *
      FROM cld_tmp_ins
     WHERE oops_payer_id IS NULL;
     
--Get counts of rows, claims and patients
SELECT Count(*) AS row_cnt,
       Count(Distinct patient_id) as pt_cnt,
       Count(Distinct claim_id) AS claim_cnt
  FROM tmp_no_payer;

## Patient_id, zip3, type_coverage
Impute NULL OOPS payers by identifying the nearest claim with a matching patient_id, zip3, and type_coverage.

In [ ]:
%%read_sql
/*Identify how many days away from a claim without an `oops_payer_id` is another claim with an oops_payer_id
with a matching patient_id, zip3, and type_coverage*/
DROP TABLE IF EXISTS tmp_nearest_delta;
CREATE TRANSIENT TABLE tmp_nearest_delta AS
    SELECT nopay.patient_id,
           nopay.claim_id,
           nopay.type_coverage,
           nopay.year_of_service,
           Min(Abs(Datediff(d,nopay.year_of_service,ins.year_of_service))) AS days_delta
      FROM cld_tmp_ins ins
           JOIN tmp_no_payer nopay
             ON nopay.patient_id = ins.patient_id
                AND nopay.member_adr_zip = ins.member_adr_zip
                AND nopay.type_coverage = ins.type_coverage
     WHERE ins.oops_payer_id IS NOT NULL
     GROUP BY nopay.patient_id, nopay.claim_id, nopay.type_coverage, nopay.year_of_service;

--Determine how many claims are close
SELECT Count(*) AS row_cnt,
       Count(Distinct patient_id) as pt_cnt,
       Count(Distinct claim_id) AS claim_cnt,
       Count(Distinct patient_id, claim_id, type_coverage) AS combo_cnt
  FROM tmp_nearest_delta;

In [ ]:
%%read_sql
/*Identiy the claim date of a claim with an oops_id that is the minimum
number of days away from the claim without an oops_id.  In case of a tie
pick the earlier claim*/
DROP TABLE IF EXISTS temp_nearest_date;
CREATE TRANSIENT TABLE temp_nearest_date AS
    SELECT delta.patient_id,
           delta.claim_id,
           delta.type_coverage,
           Min(ins.year_of_service) AS nearest_claim_dt
      FROM tmp_nearest_delta delta
           JOIN cld_tmp_ins ins
             ON delta.patient_id = ins.patient_id
                AND delta.type_coverage = ins.type_coverage
                AND Abs(Datediff(d,delta.year_of_service,ins.year_of_service)) = delta.days_delta
     WHERE ins.oops_payer_id IS NOT NULL
     GROUP BY delta.patient_id, delta.claim_id, delta.type_coverage;

In [ ]:
%%read_sql
/*Confirm counts of rows, patients, and claim counts makes sense*/
SELECT Count(*) AS row_cnt,
       Count(Distinct patient_id) AS pt_cnt,
       Count(Distinct claim_id) AS claim_cnt
  FROM temp_nearest_date;

In [ ]:
%%read_sql
--Identify the oops_id of the nearest claim with a non null oops_id
DROP TABLE IF EXISTS tmp_nearest_claim;
CREATE TRANSIENT TABLE tmp_nearest_claim AS 
  SELECT nearest.patient_id, 
         nearest.claim_id, 
         nearest.type_coverage,
         ins.oops_payer_id
    FROM temp_nearest_date nearest 
         JOIN cld_tmp_ins ins 
           ON nearest.patient_id = ins.patient_id 
              AND nearest.nearest_claim_dt = ins.year_of_service 
   WHERE ins.oops_payer_id IS NOT NULL
   GROUP BY nearest.patient_id, 
            nearest.claim_id, 
            nearest.type_coverage,
            ins.oops_payer_id; 

In [ ]:
%%read_sql
/*Confirm the counts of claims makes sense. The row count will be a bit higher
as there are some claims where more than one insurance and they will be counted
as such*/
SELECT Count(*) AS row_cnt,
       Count(Distinct patient_id) AS pt_cnt,
       Count(Distinct claim_id)
  FROM tmp_nearest_claim;

### Output
Create a new table with the imputed oops_id with identical columns to the other insurance table.  Later they will all be merged together.

In [ ]:
%%read_sql
--Create oops lookup table
CREATE OR REPLACE TEMP TABLE tmp_oops_unique AS
    SELECT DISTINCT oops_payer_id,
                    oops_payer_name,
                    oops_parent_id,
                    oops_parent_name,
                    oops_google_parent_id,
                    oops_google_parent_name
      FROM project_analytics.payer_mastering.oops_payer_xwalk;

--CREATE TRANSIENT TABLE of imputed claims
DROP TABLE IF EXISTS tmp_imput_id_zip_type;
CREATE TRANSIENT TABLE tmp_imput_id_zip_type AS
    SELECT pay.patient_id,
           pay.claim_id,
           pay.claim_number,
           pay.payer_sequence,
           pay.member_adr_zip,
           pay.member_adr_state,
           pay.year_of_service,
           pay.data_source,
           pay.payer_id,
           pay.payer_name,
           pay.type_coverage,
           pay.claim_type,
           pay.ins_group,
           pay.ins_desc,
           pay.clean_raw_payer_name,
           near.oops_payer_id,
           oops.oops_payer_name,
           oops.oops_parent_id,
           oops.oops_parent_name,
           oops.oops_google_parent_id,
           oops.oops_google_parent_name,
           pay.update_payer,
           'imput patient_id, zip, type' AS oops_update,
           NULL AS multiple_payer_flag,
           1 AS impute_flag
      FROM tmp_no_payer pay
           JOIN tmp_nearest_claim near
             ON pay.patient_id = near.patient_id
                AND pay.claim_id = near.claim_id
                AND pay.type_coverage = near.type_coverage
           JOIN tmp_oops_unique oops
             ON oops.oops_payer_id = near.oops_payer_id

In [ ]:
%%read_sql
SELECT Count(*) AS row_cnt,
       Count(Distinct patient_id) AS pt_cnt,
       Count(Distinct claim_id) AS claim_cnt
  FROM tmp_imput_id_zip_type

# Final Table
Pull together all insurance data and imputed claims into a single final table

In [ ]:
%%read_sql

--Pull together all non imputed claims with imputed claims
DROP TABLE IF EXISTS cld_med_ins;
CREATE TRANSIENT TABLE cld_med_ins AS
    --Non imputed claims
    SELECT *
      FROM cld_tmp_ins
     WHERE Not(claim_id IN (SELECT claim_id
                              FROM tmp_imput_id_zip_type))
     UNION 
    --Imputed claims by matching patient_id, zip3, and coverage_type 
    SELECT *
      FROM tmp_imput_id_zip_type;

## Multiple payer flag
Identify claims with multiple payers

In [ ]:
%%read_sql
--update multiple payer flag
CREATE OR REPLACE TEMP TABLE tmp_payer_cnt AS
    SELECT patient_id,
           claim_id,
           Count(Distinct oops_payer_id) AS payer_cnt
      FROM cld_med_ins
     GROUP BY patient_id, claim_id;

--Identify claims with more than 1 distinct payer
BEGIN;
UPDATE cld_med_ins ins
   SET ins.multiple_payer_flag = CASE WHEN payer_cnt > 1 THEN 1 ELSE 0 END
  FROM tmp_payer_cnt pay
 WHERE ins.patient_id = pay.patient_id
       AND ins.claim_id = pay.claim_id;

COMMIT;

In [ ]:
%%read_sql
--Check on the imputed claim counts
SELECT multiple_payer_flag,
       Count(*) AS cnt,
       Count(*) / (SELECT Count(*)
                     FROM cld_tmp_ins
                    WHERE oops_payer_id IS NOT NULL) AS pct
  FROM cld_med_ins
 WHERE oops_payer_id IS NOT NULL
 GROUP BY multiple_payer_flag;

In [ ]:
%%read_sql
--Summary stats for final table
SELECT data_source,
       oops_update,
       Count(Distinct claim_id) AS cnt,
       Count(Distinct claim_id) / (SELECT Count(Distinct claim_id)
                                     FROM cld_tmp_ins
                                    WHERE data_source = ins.data_source) AS share,
       Count(Distinct claim_id) / (SELECT Count(Distinct claim_id)
                                     FROM cld_tmp_ins) AS pct
       
  FROM cld_med_ins ins
 GROUP BY data_source, oops_update
 ORDER BY data_source, pct desc

# Delete tables
Delete intermediate tables

In [ ]:
%%read_sql
--Clear out unnecessary tables
DROP TABLE IF EXISTS cld_tmp_ins;
DROP TABLE IF EXISTS tmp_no_payer;
DROP TABLE IF EXISTS tmp_nearest_delta;
DROP TABLE IF EXISTS tmp_no_payer;
DROP TABLE IF EXISTS tmp_nearest_delta;
DROP TABLE IF EXISTS temp_nearest_date;
DROP TABLE IF EXISTS tmp_nearest_claim;
DROP TABLE IF EXISTS tmp_imput_id_zip_type;